In [1]:
using WordNet
using AdaGram
using AdaGramCompat

using CorpusLoaders
using CorpusLoaders.Semcor
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators
using LightXML
using JLD

In [2]:
const WN_PATH="data/corpora/WordNet-2.1/"
#WN_PATH3 = "/usr/share/nltk_data/corpora/wordnet/"
db = DB(WN_PATH)

WordNet.DB

In [3]:
#am = AdaGramCompat.AdaGramModel(load_model("models/adagram/v1_d100.adagram_model")...)
#am = AdaGramModel(load_model("models/adagram/more_senses.adagram_model")...)
#am = open(deserialize,"models/adagram/more_senses.adagram_model.jsz", "r");
am = load("models/adagram/more_senses.adagram_model.jld", "am");


In [17]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld");
ee = load("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld", "ee")

#ee = restore("models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_50__m170000000.model");

In [4]:
using CorpusLoaders.Semeval2007t7

challenges = load_challenges_semeval2007t7("data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml");
solutions = load_solutions_semeval2007t7("data/corpora/wsd/semeval2007_t7/key/dataset21.test.key");

In [6]:
semcor = index_semcor(lazyload_semcor("data/corpora/semcor2.1/brown1/tagfiles/"));


In [7]:
only_of_pos(data, pos) = filter(d->d.pos==pos, data)

only_of_pos (generic function with 1 method)

In [8]:
function sense_frequency(ss::Synset, lem::Lemma)
    lem_count = lem.tagsense_count
    #lem_count+0.1(sum(values(ss.word_counts))-lem_count) #Do not use Synset's other counts. It owrks out worse
    lem_count
end

sense_frequency (generic function with 1 method)

In [ ]:
typealias Usages{S<:AbstractString, V<:AbstractVector{S}} Dict{Synset, Abs }

In [119]:
"Collect up the usages from a indexed tagged source"
function get_usages(usage_index::Dict{String, Vector{Semcor.TaggedSentence}}, key)
    if haskey(usage_index, key)
        [map(lowercase, strip_tags(sent)) for sent in usage_index[key]]
    else
        Vector{String}[]
    end
end

function get_usages(synset::Synset)
    gloss::Vector{SubString{String}} = map(lowercase, punctuation_space_tokenize(synset.gloss))
    [gloss]
end

function get_all_usages(wn::DB, semcor::Dict{String, Vector{Semcor.TaggedSentence}}, lemma_word, pos)   
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
    
    Dict{Synset,AbstractVector{AbstractVector}}(
        (synset => [get_usages(semcor, sensekey(db, synset, lemma)); get_usages(synset)]
                    for synset in target_synsets)...)  
    
end

get_all_usages (generic function with 1 method)

In [120]:
function all_word_sense_vectors(ee::WordSenseEmbedding, word)
    get(ee.embedding, word, Vector{Float32}[])
end

function all_word_sense_vectors(am::AdaGramCompat.AdaGramModel, word)
    if haskey(am.dict.word2id, word)
        wsv_mat = word_sense_vectors(am, word)
        [view(wsv_mat,:,ii) for ii in 1:size(wsv_mat,2)]
    else
        Vector{Float32}[]
    end
end



all_word_sense_vectors (generic function with 2 methods)

In [155]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end


function synthesize_embedding(ee,context::AbstractVector, word_or_phrase::AbstractString)
    words = split(word_or_phrase, " ")
    wvs = vcat((all_word_sense_vectors(ee,w) for w in words)...)
    if length(wvs) == 0
            throw(KeyError("None of $words have embeddings"))
    end
    logprobs = [Query.logprob_of_context(ee, context, wv; skip_oov=true, normalise_over_length=true) for wv in wvs]
    weighted_average(logprobs, wvs)
end


all_identical(col) = length(col)==1 || !any(x->x!=col[1],col[2:end])


    

function _lexically_informed_embeddings(wn::DB,ee, word,lemma_word, pos)
    indexed_corpus = semcor #TODO: Pass this as a parameter, not as a gloel
    target_synset_examples = get_all_usages(wn, indexed_corpus, lemma_word, pos)
    target_synsets = collect(keys(target_synset_examples))
    lemma = db[lemma_word, pos]
        
    embeddings = Vector{Vector{Float32}}(length(target_synsets))
    for (ii,(synset, examples)) in enumerate(target_synset_examples)
        embeddings[ii] = mean(examples) do eg#::AbstractVector
        context::Vector{SubString{String}} = filter(w::AbstractString->w!=word, eg)
        synthesize_embedding(ee,context, word)
        end
    end
        
    
    if all_identical(embeddings)
        #Use MCWS
        score, index = findmax(sense_frequency(ss,lemma) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
        embeddings=[embeddings[1]]
    end
        
    embeddings,target_synsets,lemma
end

lexically_informed_embeddings(wn::DB, ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)
        
_li_embeddings = Dict{Tuple{DB, Any, String, String, Char}, Tuple{Vector{Vector{Float32}}, Vector{Synset}, Lemma}}()
function lexically_informed_embeddings(wn::DB, ee, word,lemma_word, pos)
    get!(_li_embeddings, (wn, ee, word, lemma_word, pos)) do
        _lexically_informed_embeddings(wn, ee, word, lemma_word, pos)
    end
end

lexically_informed_embeddings (generic function with 2 methods)

In [156]:
function supervised_wsd(challenge, ee, wn::DB)
    try
        embeddings,target_synsets,lemma = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        #sense_index = Query.WSD(ee, embeddings, challenge.context; skip_oov=true)
        logprobs_of_context = [logprob_of_context(ee,challenge.context, wv; skip_oov=true, normalise_over_length=true) 
                                    for wv in embeddings] 
      
        sense_index= indmax(logprobs_of_context)

        
        synset = target_synsets[sense_index]
        sk = sensekey(wn, synset, lemma)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end

function most_frequent_sense(challenge, wn::DB)
    try
        lemma = wn[challenge.pos, challenge.lemma]
        target_synsets::Vector{Synset} = synsets(db, lemma)
        
        sense_freqs =  Float32[sense_frequency(ss,lemma) for ss in target_synsets]      
        sense_index= indmax(sense_freqs)
        synset = target_synsets[sense_index]
        
        sk = sensekey(wn, synset,  lemma)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end



most_frequent_sense (generic function with 1 method)

In [161]:
using ProgressMeter

In [162]:
function evalute_on_wsd_challenge(challenges, solutions, method)
    correct = 0
    incorrect = 0
    notattempted = 0
    @showprogress for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = method(challenge)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end
    
    
function evalute_on_wsd_challenge(challenges, solutions, ee, wn::DB)
    method = challenge -> supervised_wsd(challenge, ee, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end


function evalute_on_wsd_challenge_MFS(challenges, solutions, wn::DB)
    method = challenge -> most_frequent_sense(challenge, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

evalute_on_wsd_challenge_MFS (generic function with 1 method)

In [165]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, am, db))

#println("only nouns  : \t\t", 
@show nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), am, db)
#println("only verbs  : \t\t", 
@show vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), am, db)
#println("only adjecti: \t\t", 
@show aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), am, db)
#println("only adverbs: \t\t", 
@show rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), am, db)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:11
overall: 		(0.6392935982339956,0.6381665932128691,0.6387295985884428)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
nn = evalute_on_wsd_challenge(only_of_pos(challenges,'n'),only_of_pos(solutions,'n'),am,db) = (0.6495031616982837,0.6489169675090253,0.6492099322799098)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:05
vv = evalute_on_wsd_challenge(only_of_pos(challenges,'v'),only_of_pos(solutions,'v'),am,db) = (0.6016949152542372,0.6006768189509306,0.6011854360711262)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
aa = evalute_on_wsd_challenge(only_of_pos(challenges,'a'),only_of_pos(solutions,'a'),am,db) = (0.6232686980609419,0.6215469613259669,0.6224066390041495)
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
rr = evalute_on_wsd_challenge(only_of_pos(challenges,'r'),only_of_pos(solutions,'r'),am,db) = (0.71980676328502

(0.7198067632850241,0.7163461538461539,0.7180722891566265)

In [65]:
?ccall

search: ccall HierarchicalSoftmaxNode AbstractChannel



```
ccall((symbol, library) or function_pointer, ReturnType, (ArgumentType1, ...), ArgumentValue1, ...)
```

Call function in C-exported shared library, specified by `(function name, library)` tuple, where each component is a string or symbol.

Note that the argument type tuple must be a literal tuple, and not a tuple-valued variable or expression. Alternatively, `ccall` may also be used to call a function pointer, such as one returned by `dlsym`.

Each `ArgumentValue` to the `ccall` will be converted to the corresponding `ArgumentType`, by automatic insertion of calls to `unsafe_convert(ArgumentType, cconvert(ArgumentType, ArgumentValue))`. (See also the documentation for each of these functions for further details.) In most cases, this simply results in a call to `convert(ArgumentType, ArgumentValue)`.


In [154]:
a = Dict{Float64, Int64}
length(a)


LoadError: LoadError: MethodError: no method matching length(::Type{Dict{Float64,Int64}})
Closest candidates are:
  length(!Matched::SimpleVector) at essentials.jl:168
  length(!Matched::Base.MethodList) at reflection.jl:256
  length(!Matched::MethodTable) at reflection.jl:322
  ...
while loading In[154], in expression starting on line 2

In [18]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db))

overall: 		(0.6486486486486487,0.6346408109299251,0.6415682780129205)
only nouns  : 		(0.6593001841620626,0.6462093862815884,0.6526891522333637)
only verbs  : 		(0.5785837651122625,0.5668358714043993,0.5726495726495726)
only adjecti: 		(0.6762177650429799,0.6519337016574586,0.6638537271448665)
only adverbs: 		(0.7427184466019418,0.7355769230769231,0.7391304347826086)


In [19]:
println("overall: \t\t", evalute_on_wsd_challenge_MFS(challenges, solutions, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), db))
println("only verbs  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), db))
println("only adjecti: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), db))
println("only adverbs: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), db))


overall: 		(0.7783164389598942,0.7783164389598942,0.7783164389598942)
only nouns  : 		(0.7653429602888087,0.7653429602888087,0.7653429602888087)
only verbs  : 		(0.7529610829103215,0.7529610829103215,0.7529610829103215)
only adjecti: 		(0.8038674033149171,0.8038674033149171,0.8038674033149171)
only adverbs: 		(0.875,0.875,0.875)


In [21]:
#Zero shot WSI for 1 shot WSD
using Training

In [41]:
rr = FixedWordSenseEmbedding(ee.dimension, random_inited, huffman_tree; initial_nsenses=50)
rr.corpus_size = ee.corpus_size
rr.distribution = ee.distribution
rr.codebook = ee.codebook
rr.classification_tree = ee.classification_tree
Training.initialize_embedding(rr);

In [42]:
rr.embedding["us"] |> length

50

In [43]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, rr, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), rr, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), rr, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), rr, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), rr, db))

overall: 		(0.6522522522522523,0.6381665932128691,0.6451325462241033)
only nouns  : 		(0.6869244935543278,0.6732851985559567,0.6800364630811304)
only verbs  : 		(0.5630397236614854,0.5516074450084603,0.5572649572649573)
only adjecti: 		(0.667621776504298,0.643646408839779,0.6554149085794655)
only adverbs: 		(0.6941747572815534,0.6875,0.6908212560386474)


In [44]:
hh = deepcopy(rr)
for word in keys(rr.embedding)
    append!(hh.embedding[word], ee.embedding[word])
end

In [45]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, hh, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), hh, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), hh, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), hh, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), hh, db))

overall: 		(0.654054054054054,0.6399294843543412,0.6469146803296949)
only nouns  : 		(0.6804788213627992,0.6669675090252708,0.6736554238833182)
only verbs  : 		(0.5544041450777202,0.5431472081218274,0.5487179487179487)
only adjecti: 		(0.6962750716332379,0.6712707182320442,0.6835443037974684)
only adverbs: 		(0.7233009708737864,0.7163461538461539,0.7198067632850241)


In [ ]:
using JLD